## Import Package and Read Data

In [4]:
import nltk
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string
from tqdm import tqdm

In [12]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from textblob import TextBlob

In [2]:
import pandas as pd
import gzip
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Digital_Music.json.gz')

# df.to_csv('reviews_Digital_Music_5.csv', index=True, sep=',')
# music_df = pd.read_csv('reviews_Digital_Music_5.csv')

### The review dataset looks like as the following : 

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2EFCYXHNK06IS,5555991584,"Abigail Perkins ""Abby &#34;Reads Too Much&#34...","[4, 5]",The anthemic title track begins &quot;The Memo...,5.0,Enya Experiments And Succeeds,978480000,"01 3, 2001"
1,A1WR23ER5HMAA9,5555991584,AKB,"[1, 1]",Just when I thought Enya couldn't possibly get...,5.0,How to improve upon perfection.,953424000,"03 19, 2000"
2,A2IR4Q0GPAFJKW,5555991584,Alexander,"[0, 0]",Nice CD for easy listening. My husband and I ...,4.0,Good listening.,1393545600,"02 28, 2014"
3,A2V0KUVAB9HSYO,5555991584,Alison Hight,"[0, 1]","I really liked this CD, especially the differe...",4.0,Loved It,966124800,"08 13, 2000"
4,A1J0GL9HCA7ELW,5555991584,"Al the Pal ""Al the Pal""","[3, 3]",Enya's richly chorded style has smitten me onc...,5.0,Another Hauntingly Beautiful Collection of Songs,1007683200,"12 7, 2001"


### Clean the text data

In [5]:
def text_filter(text):
    text = str(text).lower()
    text = ''.join(ch for ch in text if ch not in set(string.punctuation))  # remove punctuation
    text = re.sub(r'\d+', '', text)    # remove digits
    text = " ".join(text.split())     # remove white space
    stop_words = set(stopwords.words('english'))
    tokens1 = word_tokenize(text)
    text1 = [i for i in tokens1 if not i in stop_words] # remove stop words
    text1 = ' '.join(text1) # Back to string
    lemmatizer=WordNetLemmatizer()
    tokens2 = word_tokenize(text1)
    text2 = [lemmatizer.lemmatize(word) for word in tokens2] # lemmatizer    
    return text2

In [7]:
list_review = list(df['reviewText']) 

In [8]:
df['reviewText'] = [text_filter(x) for x in tqdm(list_review)]

100%|██████████| 836006/836006 [26:09<00:00, 532.75it/s]  


### After clean the text feature, it eliminate the redundant parts

In [9]:
df.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2EFCYXHNK06IS,5555991584,"Abigail Perkins ""Abby &#34;Reads Too Much&#34...","[4, 5]","[anthemic, title, track, begin, quotthe, memor...",5.0,Enya Experiments And Succeeds,978480000,"01 3, 2001"
1,A1WR23ER5HMAA9,5555991584,AKB,"[1, 1]","[thought, enya, couldnt, possibly, get, better...",5.0,How to improve upon perfection.,953424000,"03 19, 2000"
2,A2IR4Q0GPAFJKW,5555991584,Alexander,"[0, 0]","[nice, cd, easy, listening, husband, like, qui...",4.0,Good listening.,1393545600,"02 28, 2014"
3,A2V0KUVAB9HSYO,5555991584,Alison Hight,"[0, 1]","[really, liked, cd, especially, different, lan...",4.0,Loved It,966124800,"08 13, 2000"
4,A1J0GL9HCA7ELW,5555991584,"Al the Pal ""Al the Pal""","[3, 3]","[enyas, richly, chorded, style, smitten, littl...",5.0,Another Hauntingly Beautiful Collection of Songs,1007683200,"12 7, 2001"
5,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","[hard, believe, memory, tree, came, year, agoi...",5.0,Enya's last great album,1158019200,"09 12, 2006"
6,A340XJYJDFSMUG,5555991584,"Amazon Customer ""Arlu""","[1, 3]","[enya, fan, like, cd, cd, skip, song, song, st...",3.0,Something Different,1190419200,"09 22, 2007"
7,A3Q1J7VFGG80EK,5555991584,Amber,"[0, 0]","[im, huge, know, enya, fan, like, music, much,...",5.0,Memory of Trees or My favorite Enya CD,975628800,"12 1, 2000"
8,A1REP2FMPOXV4A,5555991584,Andrew G.,"[6, 8]","[favorite, enya, album, date, even, writing, r...",5.0,"In a word, perfection",993427200,"06 25, 2001"
9,A3QEKUPBPQ7A2S,5555991584,Archer,"[1, 1]","[love, love, love, love, love, memory, tree, e...",5.0,I could listen to this album all day!,1055635200,"06 15, 2003"


In [10]:
df.to_csv("df_clean1.csv", header = True) ##Export to csv for future use

________________________

In [11]:
df_c1 = df

In [19]:
list_review1 = list(df_c1['reviewText']) 
df_c1['reviewText'] = [" ".join(x) for x in tqdm(list_review1)]

100%|██████████| 836006/836006 [00:06<00:00, 123409.00it/s]


### Export the cleaned data for word2vec part :

In [27]:
df_c1.to_csv("df_clean2.csv", header = True) 

In [14]:
df_c1.isnull().sum()

reviewerID           0
asin                 0
reviewerName      2420
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [21]:
df_c1['reviewText'][0:10]

0    anthemic title track begin quotthe memory tree...
1    thought enya couldnt possibly get better found...
2    nice cd easy listening husband like quite bit ...
3    really liked cd especially different language ...
4    enyas richly chorded style smitten little bit ...
5    hard believe memory tree came year agoit held ...
6    enya fan like cd cd skip song song stick like ...
7    im huge know enya fan like music much cd ive h...
8    favorite enya album date even writing review c...
9    love love love love love memory tree enyaespec...
Name: reviewText, dtype: object

### Check the word frequency in text review feature, and remove the low frequency (<5) and high frequency (>1000) 

In [25]:
word_vectorizer = CountVectorizer(analyzer='word', stop_words='english', min_df=8, max_df= 1000)
Smatrix = word_vectorizer.fit_transform(df_c1['reviewText'])
frequencies = sum(Smatrix).toarray()[0]
freq_names = word_vectorizer.get_feature_names()
freq = pd.DataFrame(frequencies, index = freq_names, columns=['frequency'])
freq

,frequency
aa,215
aaa,104
aaaa,37
aaaaa,24
aaaah,8
aaah,17
aac,21
aacute,8
aacutelbum,92
aad,15


##### Create a proper frequency word bank

In [26]:
proper_freq_word_abs = set(freq_names)

In [43]:
list_review2 = list(df_c1['reviewText']) 

### Remove the low / high frequency word by compare with the word bank created before

In [46]:
def delet_extreme_fre(df, proper_freq_word):
    for index, each in  enumerate(list_review2):
        df_c2['reviewText'][index] = " ".join(set(each.split()) & proper_freq_word)       
    return df_c2

In [47]:
clean2_df = delet_extreme_fre(df_c2, proper_freq_word_abs)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### After cleaned, the text review cleaning part is finished

In [48]:
clean2_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2EFCYXHNK06IS,5555991584,"Abigail Perkins ""Abby &#34;Reads Too Much&#34...","[4, 5]",clannad flowquot frighteningly quotchina trees...,5.0,Enya Experiments And Succeeds,978480000,"01 3, 2001"
1,A1WR23ER5HMAA9,5555991584,AKB,"[1, 1]",enya treesquot conveys enyas,5.0,How to improve upon perfection.,953424000,"03 19, 2000"
2,A2IR4Q0GPAFJKW,5555991584,Alexander,"[0, 0]",,4.0,Good listening.,1393545600,"02 28, 2014"
3,A2V0KUVAB9HSYO,5555991584,Alison Hight,"[0, 1]",particularily enyas,4.0,Loved It,966124800,"08 13, 2000"
4,A1J0GL9HCA7ELW,5555991584,"Al the Pal ""Al the Pal""","[3, 3]",massage enya smitten recliner perfectlythis ex...,5.0,Another Hauntingly Beautiful Collection of Songs,1007683200,"12 7, 2001"


In [49]:
clean2_df = clean2_df[(clean2_df['reviewText'].isna() == False)].reset_index(drop=True)

-------------

### Polarity Analysis by using Textblob

#### For each review, we will have a polarity score, and we will consider that score into final score of each product

In [50]:
from textblob import TextBlob

### By using this package, each text review will be convert to a positive, negetive or neutral score.

In [53]:
a = clean2_df['reviewText'][0]
b = TextBlob(a).sentiment[0]
c = TextBlob(a).sentiment[1]
print(b)
print(c)

-0.06
0.8


In [54]:
X_score = pd.DataFrame()

In [55]:
# Polarity Analysis for Reviews
X_score['review_polarity'] = clean2_df['reviewText'].apply(lambda x: TextBlob(x).sentiment[0])
X_score['review_subjectivity'] = clean2_df['reviewText'].apply(lambda x: TextBlob(x).sentiment[1])


In [61]:
X_score.head(2)

,review_polarity,review_subjectivity
0,-0.06,0.8
1,0.00,0.0


### Write out csv file for using to build a new score for each product.

In [62]:
X_score.to_csv("Polarity.csv", header = True)